# MLP Gradient Descent

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/hanchen92/THU-STAT-Deep-113-2-6189/blob/main/03_01_mlp_gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/hanchen92/THU-STAT-Deep-113-2-6189/main/03_01_mlp_gradient_descent.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots

import plotly.graph_objects as go

$$
\begin{align*}
y_i = 1 + 2x_{i1}^2 + 3x_{i2}^2+\epsilon_i,\quad i=1,\dots,n
\end{align*}
$$

In [ ]:
np.random.seed(20250303)
n = 200

X = np.random.randn(n,2)

y = 1 + 2*(X[:,[0]])**2 + 3*(X[:,[1]])**2 + np.random.randn(n,1)

Plot the 3D surface of the function

In [ ]:
tmp_x1,tmp_x2 = np.meshgrid(np.arange(-3, 3, 0.01),np.arange(-3, 3, 0.01))

tmp_y = 1 + 2*(tmp_x1)**2 + 3*(tmp_x2)**2 

In [ ]:
fig = go.Figure(data=[go.Surface(x=tmp_x1, y=tmp_x2, z=tmp_y)])


fig.add_trace(go.Scatter3d(
    x=X[:,0], y=X[:,1], z=y.flatten(), 
    mode='markers',
    marker=dict(size=5, color='red', symbol='circle')
))



# Update layout for better viewing
fig.update_layout(
    title='3D Surface Plot',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    ),
    autosize=True,  # Set autosize to False to define fixed size
    width=800,      # Width of the figure in pixels
    height=800,     # Height of the figure in pixels
)

# Show the plot
fig.show()

# Stochastic gradient descent

In [ ]:
def my_relu(x):
    return(np.maximum(0,x))

def my_relu_gradient(x):
    return(np.where(x>0,1,0))

The MLP model is coded as the following equations:

$$
\begin{align*}
a^{(1)}&=W^{(1)}x+b^{(1)}\\
z^{(1)}&=h_1(a^{(1)})\\
a^{(2)}&=W^{(2)}z^{(1)}+b^{(2)}\\
z^{(2)}&=h_1(a^{(2)})\\
a^{(3)}&=W^{(3)}z^{(2)}+b^{(3)}
\end{align*}
$$


In [ ]:
input_n = 2
hidden_n = [10,10]
output_n = 1


def stochastic_gradient_descent():
    n_epcho = 1000
    learning_rate = 0.01

    np.random.seed(20250226)
    W1 = np.random.randn(hidden_n[0],input_n)*0.1
    b1 = np.zeros((hidden_n[0],1))
    
    W2 = np.random.randn(hidden_n[1],hidden_n[0])*0.1
    b2 = np.zeros((hidden_n[1],1))
    
    W3 = np.random.randn(output_n,hidden_n[1])*0.1
    b3 = np.zeros((output_n,1))

    for tmp_epcho in range(n_epcho):
        for i in range(n):
            tmp_x = X.T[:,[i]]
            tmp_y = y[[i]]
            
            a1 = W1@tmp_x + b1 
            z1 = my_relu(a1)
    
    
    
            a2 = W2@z1 + b2
            z2 = my_relu(a2)
    
            
            a3 = W3@z2 + b3
    
            dL_da3 = a3-tmp_y
            dL_da2 = (W3.T@dL_da3)*my_relu_gradient(a2)
            dL_da1 = (W2.T@dL_da2)*my_relu_gradient(a1)

            
            dL_W3 = dL_da3@z2.T
            dL_b3 = dL_da3
            
            dL_W2 = dL_da2@z1.T
            dL_b2 = dL_da2

            dL_W1 = dL_da1@tmp_x.T
            dL_b1 = dL_da1

            W3 = W3 - learning_rate*dL_W3
            b3 = b3 - learning_rate*dL_b3

            W2 = W2 - learning_rate*dL_W2
            b2 = b2 - learning_rate*dL_b2

            W1 = W1 - learning_rate*dL_W1
            b1 = b1 - learning_rate*dL_b1

        all_a1 = W1@ X.T + b1 
        all_z1 = my_relu(all_a1)

        all_a2 = W2@all_z1 + b2
        all_z2 = my_relu(all_a2)

        
        all_a3 = W3@all_z2 + b3
        # Print the loss every 100 epochs
        if tmp_epcho % 100 == 0:
            loss = np.mean((all_a3.T - y) ** 2)
            print(f'Epoch {tmp_epcho}, Loss: {loss:.4f}')
        

    return([[W1,b1],[W2,b2],[W3,b3]])

In [ ]:
[[W1,b1],[W2,b2],[W3,b3]] = stochastic_gradient_descent()

In [ ]:
tmp_x1,tmp_x2 = np.meshgrid(np.arange(-3, 3, 0.01),np.arange(-3, 3, 0.01))

r = tmp_x1.shape[0]
c = tmp_x1.shape[1]

tmp_y_estimate = np.array([[0 for _ in range(c)] for _ in range(r)])

for i in range(r):
    for j in range(c):
        tmp_x = np.array([[tmp_x1[i,j]], [tmp_x2[i,j]]])
        a1 = W1@tmp_x + b1
        z1 = my_relu(a1)



        a2 = W2@z1 + b2
        z2 = my_relu(a2)

        
        a3 = W3@z2 + b3
        #print(a3.flatten())
        tmp_y_estimate[i,j] = a3.flatten()


In [ ]:
fig = go.Figure(data=[go.Surface(x=tmp_x1, y=tmp_x2, z=tmp_y_estimate)])


fig.add_trace(go.Scatter3d(
    x=X[:,0], y=X[:,1], z=y.flatten(), 
    mode='markers',
    marker=dict(size=5, color='red', symbol='circle')
))



# Update layout for better viewing
fig.update_layout(
    title='3D Surface Plot',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    ),
    autosize=True,  # Set autosize to False to define fixed size
    width=800,      # Width of the figure in pixels
    height=800,     # Height of the figure in pixels
)

# Show the plot
fig.show()

# Batch Gradient Descent

The MLP model is coded as the following equations:

$$
\begin{align*}
A^{(1)}&=XW^{(1)}+b^{(1)}\\
Z^{(1)}&=h_1(A^{(1)})\\
Z^{(2)}&=Z^{(1)}W^{(2)}+b^{(2)}\\
Z^{(2)}&=h_1(A^{(2)})\\
A^{(3)}&=Z^{(2)}W^{(3)}+b^{(3)}
\end{align*}
$$


In [ ]:
input_n = 2
hidden_n = [10,10]
output_n = 1


def batch_gradient_descent():
    n_epcho = 1000
    learning_rate = 0.01

    np.random.seed(20250226)
    W1 = np.random.randn(input_n,hidden_n[0])*0.1
    b1 = np.zeros((1,hidden_n[0]))
    
    W2 = np.random.randn(hidden_n[0],hidden_n[1])*0.1
    b2 = np.zeros((1,hidden_n[1]))
    
    W3 = np.random.randn(hidden_n[1],output_n)*0.1
    b3 = np.zeros((1,output_n))

    for tmp_epcho in range(n_epcho):

        a1 = X@W1+b1
        z1 = my_relu(a1)

        a2 = z1@W2+b2
        z2 = my_relu(a2)

        a3 = z2@W3+b3

        dL_da3 = a3-y
        dL_da2 = (dL_da3@W3.T)*my_relu_gradient(a2)
        dL_da1 = (dL_da2@W2.T)*my_relu_gradient(a1)

        dL_W3 = z2.T@dL_da3/n
        dL_b3 = np.sum(dL_da3,axis=0)/n
        
        dL_W2 = z1.T@dL_da2/n
        dL_b2 = np.sum(dL_da2,axis=0)/n

        dL_W1 = X.T@dL_da1/n
        dL_b1 = np.sum(dL_da1,axis=0)/n

        W3 = W3 - learning_rate*dL_W3
        b3 = b3 - learning_rate*dL_b3

        W2 = W2 - learning_rate*dL_W2
        b2 = b2 - learning_rate*dL_b2

        W1 = W1 - learning_rate*dL_W1
        b1 = b1 - learning_rate*dL_b1
        
        # Print the loss every 100 epochs
        if tmp_epcho % 100 == 0:
            loss = np.mean((a3 - y) ** 2)
            print(f'Epoch {tmp_epcho}, Loss: {loss:.4f}')
 

    return([[W1,b1],[W2,b2],[W3,b3]])

In [ ]:
[[W1,b1],[W2,b2],[W3,b3]] = batch_gradient_descent()

In [ ]:
tmp_x1,tmp_x2 = np.meshgrid(np.arange(-3, 3, 0.01),np.arange(-3, 3, 0.01))

r = tmp_x1.shape[0]
c = tmp_x1.shape[1]

tmp_y_estimate = np.array([[0 for _ in range(c)] for _ in range(r)])

for i in range(r):
    for j in range(c):
        tmp_x = np.array([tmp_x1[i,j], tmp_x2[i,j]])
        a1 = tmp_x@W1 + b1
        z1 = my_relu(a1)



        a2 = z1@W2 + b2
        z2 = my_relu(a2)

        
        a3 = z2@W3 + b3
        #print(a3.flatten())
        tmp_y_estimate[i,j] = a3.flatten()


In [ ]:
fig = go.Figure(data=[go.Surface(x=tmp_x1, y=tmp_x2, z=tmp_y_estimate)])


fig.add_trace(go.Scatter3d(
    x=X[:,0], y=X[:,1], z=y.flatten(), 
    mode='markers',
    marker=dict(size=5, color='red', symbol='circle')
))



# Update layout for better viewing
fig.update_layout(
    title='3D Surface Plot',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    ),
    autosize=True,  # Set autosize to False to define fixed size
    width=800,      # Width of the figure in pixels
    height=800,     # Height of the figure in pixels
)

# Show the plot
fig.show()